In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Paths to the CSV files
train_path = './training.csv'
test_path = './test.csv'
lookid_path = './IdLookupTable.csv'

# Read the CSV files
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
lookid_data = pd.read_csv(lookid_path)

# Columns for Y_train
y_columns = [
    'nose_tip_x', 'nose_tip_y',
    'mouth_left_corner_x', 'mouth_left_corner_y',
    'mouth_right_corner_x', 'mouth_right_corner_y',
    'mouth_center_top_lip_x', 'mouth_center_top_lip_y',
    'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y'
]

# Select the Image column for X_train
X_train = train_data['Image']

# Select the specified columns for Y_train
Y_train = train_data[y_columns]

# Concatenate X_train and Y_train to discard rows with missing values
combined_data = pd.concat([X_train, Y_train], axis=1)

# Drop rows with any missing values
combined_data = combined_data.dropna()

# Split the combined data back into X_train and Y_train
X_train = combined_data['Image']
Y_train = combined_data[y_columns]

# Convert X_train to numpy array and reshape
X_train = np.array([np.fromstring(image, sep=' ').reshape(96, 96, 1) for image in X_train])
X_train = X_train.astype('float32') / 255.0  # Normalize pixel values

# Convert Y_train to numpy array
Y_train = np.array(Y_train)

# Define the model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(96, 96, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(30, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10))

# Compile the model
model.compile(optimizer='adam', loss='mse')


# Save the model
# model.save('path/to/your/model')


ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
import pandas as pd
import numpy as np
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

# Paths to the CSV files
train_path = './train.csv'
test_path = './test.csv'
# lookid_path = './IdLookupTable.csv'

# Read the CSV files
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
# lookid_data = pd.read_csv(lookid_path)

# Columns for Y_train
y_columns = [
    'left_eye_center_x','left_eye_center_y',
    'right_eye_center_x','right_eye_center_y',
    'nose_tip_x', 'nose_tip_y',
    'mouth_left_corner_x', 'mouth_left_corner_y',
    'mouth_right_corner_x', 'mouth_right_corner_y',
    'mouth_center_top_lip_x', 'mouth_center_top_lip_y',
    'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y'
]

# Select the Image column for X_train
X_train = train_data['Image']

# Select the specified columns for Y_train
Y_train = train_data[y_columns]

# Concatenate X_train and Y_train to find rows with missing values
combined_data = pd.concat([X_train, Y_train], axis=1)

# Find rows with any missing values
missing_data = combined_data[combined_data.isnull().any(axis=1)]
non_missing_data = combined_data.dropna()

# Split the combined data back into X_train and Y_train
X_train_non_missing = non_missing_data['Image']
Y_train_non_missing = non_missing_data[y_columns]

# Convert X_train to numpy array and reshape
X_train_non_missing = np.array([np.fromstring(image, sep=' ').reshape(96, 96, 1) for image in X_train_non_missing])
X_train_non_missing = X_train_non_missing.astype('float32') / 255.0  # Normalize pixel values

# Convert Y_train to numpy array
Y_train_non_missing = np.array(Y_train_non_missing)

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

# Function to detect face landmarks
def detect_face_landmarks(image):
    results = face_mesh.process(image)
    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark
        return landmarks
    return None

# Process images with missing data
for idx, row in missing_data.iterrows():
    image = np.fromstring(row['Image'], sep=' ').reshape(96, 96)
    image_rgb = cv2.cvtColor(image.astype('uint8'), cv2.COLOR_GRAY2RGB)
# keypoint_indices = [4, 61, 291, 0, 17]  # Indices of the keypoints to extract

    landmarks = detect_face_landmarks(image_rgb)
    if landmarks:
        # Example to fill nose_tip_x and nose_tip_y
        row['nose_tip_x'] = landmarks[4].x * 96
        row['nose_tip_y'] = landmarks[4].y * 96
        row['mouth_left_corner_x'] = landmarks[61].x * 96
        row['mouth_left_corner_y'] = landmarks[61].y * 96
        row['mouth_right_corner_x'] = landmarks[291].x * 96
        row['mouth_right_corner_y'] = landmarks[291].y * 96
        row['mouth_center_top_lip_x'] = landmarks[0].x * 96
        row['mouth_center_top_lip_y'] = landmarks[0].y * 96
        row['mouth_center_bottom_lip_x'] = landmarks[17].x * 96
        row['mouth_center_bottom_lip_y'] = landmarks[17].y * 96
        row['left_eye_center_x']= landmarks[263].x * 96
        row['left_eye_center_y'] = landmarks[263].x * 96
        row['right_eye_center_x']= landmarks[33].x * 96
        row['right_eye_center_y']= landmarks[33].x * 96
        
        # Add more points as needed
        combined_data.loc[idx] = row

# Drop rows still having any missing values
combined_data = combined_data.dropna()

# Split the combined data back into X_train and Y_train
X_train = combined_data['Image']
Y_train = combined_data[y_columns]

# Convert X_train to numpy array and reshape
X_train = np.array([np.fromstring(image, sep=' ').reshape(96, 96, 1) for image in X_train])
X_train = X_train.astype('float32') / 255.0  # Normalize pixel values

# Convert Y_train to numpy array
Y_train = np.array(Y_train)

print(len(Y_train))

# Show one of the images with landmarks
def show_image_with_landmarks(image, landmarks):
    plt.imshow(image, cmap='gray')
    for landmark in landmarks:
        plt.scatter(landmark.x * image.shape[1], landmark.y * image.shape[0], s=10, c='red')
    plt.show()

# Display an example
example_idx = missing_data.index[0]
example_image = np.fromstring(missing_data.loc[example_idx, 'Image'], sep=' ').reshape(96, 96)
example_image_rgb = cv2.cvtColor(example_image.astype('uint8'), cv2.COLOR_GRAY2RGB)

example_landmarks = detect_face_landmarks(example_image_rgb)
if example_landmarks:
    show_image_with_landmarks(example_image, example_landmarks)



In [ ]:
# Train the model
history = model.fit(X_train, Y_train, epochs=350, batch_size=32, validation_split=0.2)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Process test data
test_images = np.array([np.fromstring(image, sep=' ').reshape(96, 96, 1) for image in test_data['Image']])
test_images = test_images.astype('float32') / 255.0

# Predict key points on test data
predicted_points = model.predict(test_images)

# Function to plot image with key points
def plot_image_with_keypoints(image, keypoints):
    plt.imshow(image, cmap='gray')
    keypoints = keypoints.reshape(-1, 2)
    for (x, y) in keypoints:
        plt.plot(x, y, 'ro')
    plt.show()

# Plot some random test images with predicted key points
num_images_to_show = 5
random_indices = random.sample(range(len(test_images)), num_images_to_show)

for i in random_indices:
    plot_image_with_keypoints(test_images[i].reshape(96, 96), predicted_points[i])
